<a href="https://colab.research.google.com/github/dookda/cmu_py499/blob/main/proj_knot_sentiment/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy scikit-learn
!pip install torch torchvision         # for PyTorch
!pip install tensorflow                # for TF/Keras (if preferred)
!pip install transformers              # for BERT
!pip install nltk regex emoji          # for preprocessing

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

!cp /content/drive/MyDrive/_DATASET/sentiment/tweets.csv -d /content/tweets.csv

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
import pandas as pd
df = pd.read_csv('tweets.csv')
print(df.airline_sentiment.value_counts())


airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64


In [9]:
import re, emoji
from nltk.corpus import stopwords
STOP = set(stopwords.words('english'))

def clean_tweet(s):
    s = emoji.demojize(s)                            # 😊 → :smiling_face:
    s = re.sub(r'http\S+|@\w+|#','', s.lower())      # remove URLs, mentions, hashtags
    s = re.sub(r'[^a-z0-9_: ]',' ', s)               # keep emojis codes & ascii
    tokens = [t for t in s.split() if t not in STOP]
    return ' '.join(tokens)

df['clean_text'] = df.text.apply(clean_tweet)


In [10]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df.airline_sentiment, random_state=42)


In [14]:
import os
import re
import io
import emoji
import zipfile
import urllib.request

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

# 1. Download NLTK data (run once)
nltk.download('punkt')
nltk.download('stopwords')

# 2. Load dataset
df = pd.read_csv('tweets.csv')[['text','airline_sentiment']].dropna()

# 3. Clean text
STOP = set(stopwords.words('english'))
def clean_text(s):
    s = emoji.demojize(s)                              # 😊 → :smiling_face:
    s = re.sub(r'http\S+|@\w+|#', '', s.lower())       # remove URLs, mentions, hashtags
    s = re.sub(r'[^a-z0-9_: ]', ' ', s)                # keep emoji codes & alphanum
    tokens = [t for t in s.split() if t not in STOP]
    return ' '.join(tokens)

df['clean_text'] = df['text'].map(clean_text)

# 4. Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['airline_sentiment'])

# 5. Train/validation split
train_df, val_df = train_test_split(
    df[['clean_text','label']],
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

# 6. Tokenize & pad
MAX_LEN = 50
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['clean_text'])

def to_padded(seqs):
    s = tokenizer.texts_to_sequences(seqs)
    return pad_sequences(s, maxlen=MAX_LEN, padding='post', truncating='post')

X_train, X_val = to_padded(train_df['clean_text']), to_padded(val_df['clean_text'])
y_train, y_val = train_df['label'].values, val_df['label'].values

vocab_size = len(tokenizer.word_index) + 1
EMB_DIM    = 100

# 7. Ensure GloVe file exists (download & unzip if needed)
GLOVE_FILE = 'glove.6B.100d.txt'
if not os.path.isfile(GLOVE_FILE):
    print("Downloading GloVe embeddings (862 MB zip)...")
    url = 'http://nlp.stanford.edu/data/glove.6B.zip'
    resp = urllib.request.urlopen(url)
    with zipfile.ZipFile(io.BytesIO(resp.read())) as z:
        print("Extracting GloVe 100d vectors...")
        z.extract(GLOVE_FILE)
    print("Done.")

# 8. Load GloVe into a dict
embeddings_index = {}
with open(GLOVE_FILE, 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word   = values[0]
        vec    = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = vec

# 9. Build embedding matrix
embedding_matrix = np.zeros((vocab_size, EMB_DIM))
for word, i in tokenizer.word_index.items():
    vec = embeddings_index.get(word)
    if vec is not None:
        embedding_matrix[i] = vec

# 10. Define the Bi-LSTM model
model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=EMB_DIM,
        weights=[embedding_matrix],
        input_length=MAX_LEN,
        trainable=False
    ),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

# 11. Train
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

# 12. Evaluate
y_pred = model.predict(X_val).argmax(axis=-1)
print(classification_report(
    y_val, y_pred,
    target_names=le.classes_, digits=4
))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Extracting GloVe 100d vectors...
Done.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     1,156,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,156,000 (4.41 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,156,000 (4.41 MB)

Epoch 1/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 23s 102ms/step - accuracy: 0.6477 - loss: 0.8065 - val_accuracy: 0.7428 - val_loss: 0.6198
Epoch 2/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 18s 89ms/step - accuracy: 0.7511 - loss: 0.6068 - val_accuracy: 0.7616 - val_loss: 0.5961
Epoch 3/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 20s 85ms/step - accuracy: 0.7704 - loss: 0.5705 - val_accuracy: 0.7650 - val_loss: 0.5762
Epoch 4/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.7806 - loss: 0.5471 - val_accuracy: 0.7749 - val_loss: 0.5570
Epoch 5/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 20s 85ms/step - accuracy: 0.7920 - loss: 0.5177 - val_accuracy: 0.7678 - val_loss: 0.5658
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
              precision    recall  f1-score   support

    negative     0.8440    0.8578    0.8508      1835
     neutral     0.5756    0.5774    0.5765       620
    positive     0.7166    0.6681    0.6915       473

    accuracy                         0.7678      2928
   macro avg     0.7120    0.7011    0.706

In [16]:
# 13. Test on a new tweet
def predict_sentiment(tweet):
    tweet_clean = clean_text(tweet)
    seq = tokenizer.texts_to_sequences([tweet_clean])
    padded_seq = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    pred = model.predict(padded_seq).argmax(axis=-1)
    return le.inverse_transform(pred)[0]
# Example usage
tweet = "I love flying with this airline! 😊✈️"

print(f"Sentiment: {predict_sentiment(tweet)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Sentiment: positive
